In [1]:
import re
import random
import time
import datetime

import pandas as pd
import numpy as np
import torch

from scipy import stats
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
def separator(text):
    return re.sub(pattern="<b.*/>|\.|\?|\!", string=text, repl=" [SEP] ")

In [3]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
import transformers
transformers.__version__

'4.4.0'

In [5]:
import keras
keras.__version__

'2.3.1'

In [6]:
import tensorflow
tensorflow.__version__

'2.2.0'

In [7]:
import torch
torch.__version__

'1.10.2+cu102'

In [7]:
torch.cuda.is_available()

True

In [8]:
df = pd.read_csv('total_final_cnn_0502.csv', encoding = 'utf-8', index_col = 0)
df

,Unnamed: 0.1,reviews,label,new_label,new_star
0,0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 살 걸 그렇다 라...,0,0,1
1,1,침대 프레임 정말 실망 기존 싸다 제품 비교 상품 설명 보고 수납 달리 홀 뚫리다 ...,0,0,3
2,2,별로 사진 처럼 침대 하단 부 시트 지 가다 조금 벗겨지다 기사 설치 그렇다 건 지...,0,0,1
3,3,설치 기사 가다 프레임 헤드 쪽 깨다 못 보신 건지다 말씀 안 해주다 속상하다 누우...,0,0,1
4,4,월일 설치 달 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다...,0,0,1
...,...,...,...,...,...
21304,21304,폭 아쉽다 기다 하지만 튼튼하다 분위기,1,0,4
21305,21305,사용 좋다,1,0,3
21306,21306,튼튼하다 수납 공간 좋다 여전하다 만족하다,1,0,5
21307,21307,지방 보다 배송 늦다 기사 바쁘다 그냥 집 두다 가시 보다 균형 잡다 애 먹다 후 ...,1,0,2


In [10]:
df = df[['reviews', 'new_star']]
df

,reviews,new_star
0,﻿좋다 가격 구매 수납 공간 조금 많이 작다 정도 그냥 없다 모델 살 걸 그렇다 라...,1
1,침대 프레임 정말 실망 기존 싸다 제품 비교 상품 설명 보고 수납 달리 홀 뚫리다 ...,3
2,별로 사진 처럼 침대 하단 부 시트 지 가다 조금 벗겨지다 기사 설치 그렇다 건 지...,1
3,설치 기사 가다 프레임 헤드 쪽 깨다 못 보신 건지다 말씀 안 해주다 속상하다 누우...,1
4,월일 설치 달 사용 서랍 장이 분리 버리다 서랍 장이 톱밥 뭉치다 합판 너무 약하다...,1
...,...,...
21304,폭 아쉽다 기다 하지만 튼튼하다 분위기,4
21305,사용 좋다,3
21306,튼튼하다 수납 공간 좋다 여전하다 만족하다,5
21307,지방 보다 배송 늦다 기사 바쁘다 그냥 집 두다 가시 보다 균형 잡다 애 먹다 후 ...,2


In [12]:
train_texts = df.reviews
classes = df.new_star
classes_oh = classes.values

In [13]:
def bert_tokenize(texts):
    return [tokenizer.tokenize("[CLS] " + text + " [SEP]") for text in texts]

In [14]:
tokenizer = BertTokenizer.from_pretrained('klue/bert-base', do_lower_case=False)
tokenized = bert_tokenize(train_texts)
bert_ids = [tokenizer.convert_tokens_to_ids(tokens) for tokens in tokenized]

In [15]:
number_of_tokens = np.array([len(bert_id) for bert_id in bert_ids])
stats.describe(number_of_tokens)

DescribeResult(nobs=21309, minmax=(3, 124), mean=27.42165282275095, variance=465.00809317568905, skewness=1.451969884288213, kurtosis=1.6958084056781564)

In [16]:
MAX_LEN = 300
padded_bert_ids = pad_sequences(bert_ids, maxlen=MAX_LEN, dtype='long',
									truncating='post', padding='post')
padded_bert_ids[0]


array([    2,  1560,  2062,  3852,  4625, 14272,  4101,  4027,  3732,
        1518,  2062,  3681,  4181,  1415,  2062,  4347,  1236,   572,
        3649,  2062,  9932,  1176,  4137,  5857,   572,  1381,  3788,
        1258,   543,  2062,  1415,  2062,  3959,  8427,  4530,  1381,
        2062, 20852,  4229,  6145,  2223,  2062,  9996,  1156,  9488,
        4198,  2062,  4828,  1521,  1897,  2223,  2062,  5037,  2062,
        3641,  3760,  4517,  6179,  2062,  4198,  2062,  1122,  2062,
           3,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,

In [17]:
attention_masks = []
for seq in padded_bert_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

len(attention_masks)

21309

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_bert_ids, classes_oh, random_state=42, test_size=0.2)

masks_train, masks_test, _, _ = train_test_split(attention_masks, padded_bert_ids, 
                                                 random_state=42, test_size=0.2)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=0.1)

masks_train, masks_val, _, _ = train_test_split(masks_train, masks_train, 
                                                 random_state=42, test_size=0.1)
display(
    f"X_train: {X_train.shape}",
    f"X_val: {X_val.shape}",
    f"X_test: {X_test.shape}",
    f"y_train: {y_train.shape}",
    f"y_val: {y_val.shape}",
    f"y_test: {y_test.shape}",
    f"masks_train: {len(masks_train)}",
    f"masks_val: {len(masks_val)}",
    f"masks_test: {len(masks_test)}",
)

'X_train: (15342, 300)'

'X_val: (1705, 300)'

'X_test: (4262, 300)'

'y_train: (15342,)'

'y_val: (1705,)'

'y_test: (4262,)'

'masks_train: 15342'

'masks_val: 1705'

'masks_test: 4262'

In [19]:
train_inputs = torch.tensor(X_train)
train_labels = torch.tensor(y_train)
train_masks = torch.tensor(masks_train)
validation_inputs = torch.tensor(X_val)
validation_labels = torch.tensor(y_val)
validation_masks = torch.tensor(masks_val)

test_inputs = torch.tensor(X_test)
test_labels = torch.tensor(y_test)
test_masks = torch.tensor(masks_test)

print(train_inputs.shape)
print(train_labels.shape)
print(train_masks.shape)
print(validation_inputs.shape)
print(validation_labels.shape)
print(validation_masks.shape)
print(test_inputs.shape)
print(test_labels.shape)
print(test_masks.shape)

torch.Size([15342, 300])
torch.Size([15342])
torch.Size([15342, 300])
torch.Size([1705, 300])
torch.Size([1705])
torch.Size([1705, 300])
torch.Size([4262, 300])
torch.Size([4262])
torch.Size([4262, 300])


In [20]:
BATCH_SIZE = 4

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

In [21]:
from transformers import logging

logging.set_verbosity_warning()

In [22]:
model = BertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=5)

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

epochs = 4
total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [23]:
device = torch.device("cuda:0")
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [24]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [25]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')


There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [27]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))
    

# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                

        
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
#         # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...


NameError: name 'outputs' is not defined

In [25]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

  Batch   100  of    996.    Elapsed: 0:00:07.
  Batch   200  of    996.    Elapsed: 0:00:14.
  Batch   300  of    996.    Elapsed: 0:00:21.
  Batch   400  of    996.    Elapsed: 0:00:28.
  Batch   500  of    996.    Elapsed: 0:00:34.
  Batch   600  of    996.    Elapsed: 0:00:41.
  Batch   700  of    996.    Elapsed: 0:00:48.
  Batch   800  of    996.    Elapsed: 0:00:55.
  Batch   900  of    996.    Elapsed: 0:01:02.

Accuracy: 0.9405
Test took: 0:01:09


In [26]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [27]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits


In [28]:
sentences = pd.read_csv('./(final_final)review_test_data_total.csv', encoding = 'utf-8')
sentences_re = []
for i in sentences['review']:
    # print(i)
    # print('---------------------')
    logits = test_sentences([i])
    # print(logits)

    if np.argmax(logits) == 1 :
        print("긍정")
        sentences_re.append(1)
    elif np.argmax(logits) == 0 :
        print("부정")
        sentences_re.append(0)

긍정
부정
긍정
부정
긍정
부정
긍정
부정
부정
부정
부정
긍정
긍정
부정
부정
부정
부정
부정
긍정
부정
부정
긍정
부정
부정
긍정
부정
부정
긍정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
부정
부정
부정
부정
부정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
부정
긍정
긍정
부정
긍정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
부정
부정
부정
긍정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
부정
부정
부정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
긍정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
부정
부정
긍정
긍정
부정
긍정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
부정
부정
부정
긍정
긍정
긍정
부정
긍정
부정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
긍정
부정
부정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
긍정
부정
긍정
부정
긍정
부정
긍정
긍정
부정
부정
긍정
부정
부정
긍정
긍정
부정


In [104]:
sentences_re

[1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0]

In [29]:
# 모델 예측값 Series화
pd.Series(sentences_re)

# gru_predict 컬럼 생성
sentences['BERT_predict'] = pd.Series(sentences_re)

# 일치도 확인
sentences.loc[sentences['긍(1)/부정(0)'] == sentences['BERT_predict']]

# 정확도 출력
print('BERT 예측 확률 : {}'.format(len(sentences.loc[sentences['긍(1)/부정(0)'] == sentences['BERT_predict']])/len(sentences)))

BERT 예측 확률 : 0.75


In [ ]:
# logits = test_sentences([])
# print(logits)

# if np.argmax(logits) == 0 :
#     print("부정")
    
# elif np.argmax(logits) == 1 :
#     print("긍정")    

In [57]:
sentences = pd.read_csv('./(final)review_test_data_total.csv', encoding = 'utf-8')

In [89]:
# np.argmax(test_sentences('안녕하세요'))
np.argmax(test_sentences('안녕'))

0

In [ ]:
sentences_re = []
for i in sentences['review']:
    logits = test_sentences([(i)])
    if np.argmax(logits) == 0 :
        sentences_re.append(0)
    elif np.argmax(logits) == 1 :
        sentences_re.append(1)
    # test_sentences(i)
    n += 1
    print (logits)
    # 긍부정 결과 리스트에 추가하기

In [73]:
 np.argmax(test_sentences('BERT야 왜그러니'))

16

In [91]:
#파일 불러오기
sentences = pd.read_csv('./(final)review_test_data_total.csv', encoding = 'utf-8')

# 리뷰데이터 하나씩 불러와서 확인하기
n = 1

sentences_re = []
for i in sentences['review']:
    print(n)
    print('---------------------------------------------------------------')
    print(i)
    print(test_sentences(i))
    print('------------------------------------------------------------')
    n += 1
    # 긍부정 결과 리스트에 추가하기
    # sentences_re.append(test_sentences(i))

    logits = test_sentences(i)
    if np.argmax(logits) == 0 :
        sentences_re.append(0)
    elif np.argmax(logits) == 1 :
        sentences_re.append(1)
    # # test_sentences(i)
    n += 1
    # print (sentences_re)


# 모델 예측값 Series화
pd.Series(sentences_re)

# gru_predict 컬럼 생성
sentences['BERT_predict'] = pd.Series(sentences_re)

# 일치도 확인
sentences.loc[sentences['긍(1)/부정(0)'] == sentences['BERT_predict']]

# 정확도 출력
print('BERT 예측 확률 : {}'.format(len(sentences.loc[sentences['긍(1)/부정(0)'] == sentences['BERT_predict']])/len(sentences)))

1
---------------------------------------------------------------
이것저것 많이 올려놓을 수 있어서 좋고 내장된 콘센트도 좋아요 수납도 총 5칸 있어서 편해요 근데 예전 수납 침대에는 매트리스 아래 넓은 부분에 캐리어를 넣어놨는데 요 프레임은 높이(깊이)가 낮아서 캐리어 넣고 안 닫히더라구요 그점이 좀 아쉽지만 만족합니다 배송도 제가 원하는 날짜에 맞춰주셨고 기사님도 친절하고 좋았어요


TypeError: new(): invalid data type 'str'

In [40]:
print(np.argmax(logits))

262


In [44]:
outputs

SequenceClassifierOutput(loss=None, logits=tensor([[ 5.5642, -5.6210]], device='cuda:0'), hidden_states=None, attentions=None)